## **Function Interface**

`Function` module: allows you to put any code you want into an empty node, which you then can put in your workflow exactly where it needs to be!



## **A Simple Function Interface**

The most common way will involve functions you code yourself as part of your Nipype scripts.

In [1]:
def add_two(x_input):
    return x_input + 2

# This simple function takes a value, adds 2 to it,
# and returns that new value.

Just as Nipype interfaces have inputs and outputs, Python functions have inputs, in the form of parameters or arguments, and outputs, in the form of their return values. When you define a Function interface object with an existing function, as in the case of add_two() above, you must pass the constructor info about the function's inputs, its outputs, and the function itself.

In [7]:
from nipype import Node, Function

addtwo = Node(Function(input_names=["x_input"],
                       output_names=["val_output"],
                       function=add_two),
              name='add_node')

In [9]:
addtwo.inputs.x_input= 4
addtwo.run()

190204-03:34:00,883 nipype.workflow INFO:
	 [Node] Setting-up "add_node" in "/tmp/tmpybf6ktb_/add_node".
190204-03:34:00,902 nipype.workflow INFO:
	 [Node] Running "add_node" ("nipype.interfaces.utility.wrappers.Function")
190204-03:34:00,926 nipype.workflow INFO:
	 [Node] Finished "add_node".


In [10]:
addtwo.run().outputs

190204-03:34:12,961 nipype.workflow INFO:
	 [Node] Setting-up "add_node" in "/tmp/tmpybf6ktb_/add_node".
190204-03:34:12,975 nipype.workflow INFO:
	 [Node] Cached "add_node" - collecting precomputed outputs
190204-03:34:12,985 nipype.workflow INFO:
	 [Node] "add_node" found cached.



val_output = 6

In [11]:
addtwo.result.outputs


val_output = 6

You need to be careful that the name of the input parameter to the node is the same name as the input parmeter to the function, i.e. `x-input`. But you don't have to specify input_names or output names. 

In [12]:
addtwo = Node(Function(function=add_two), name="add_node")
addtwo.inputs.x_input = 8
addtwo.run()

190204-03:35:42,877 nipype.workflow INFO:
	 [Node] Setting-up "add_node" in "/tmp/tmpfsf6li9s/add_node".
190204-03:35:42,888 nipype.workflow INFO:
	 [Node] Running "add_node" ("nipype.interfaces.utility.wrappers.Function")
190204-03:35:42,904 nipype.workflow INFO:
	 [Node] Finished "add_node".


In [13]:
addtwo.result.outputs


out = 10

## **Using External Packages**

If you want to use more Python packages geared towards neuroimaging such as Nibabel, Nipy or PyMVPA, **you have to import those external functions or classes within the function itself.**

In [14]:
def get_n_trs(in_file):
    import nibabel
    f = nibabel.load(in_file)
    return f.shape[-1]

# Explicitly importing Nibabel in the body of the function!

## **`function_str` input**

This input takes not a function object, but actually **a single string that can be parsed to define a function.** In the equivalent case to our example above, the string would be

In [15]:
add_two_str = "def add_two(val):\n return val + 2\n"

OMG what is this lol ㅋㅋㅋ

## **Important - Function nodes are closed environments**

There's only one trap that you should be aware of when using the `Function` module.

If you want to use another module inside a function, you have to import it again inside the function.

In [16]:
from nipype import Node, Function

def get_random_array(array_shape):
    
    from numpy.random import random
    
    return random(array_shape)

rndArray = Node(Function(input_names=["array_shape"],
                         output_names=["random_array"],
                        
                         function=get_random_array),
                name='rndArray_node')

rndArray.inputs.array_shape = (3, 3)

rndArray.run()

print(rndArray.result.outputs)

# Function is a closed environment.
# If you want to use another module inside a function (i.e. numpy.random),
# you have to import it inside the function(local), but globally

190204-03:48:22,936 nipype.workflow INFO:
	 [Node] Setting-up "rndArray_node" in "/tmp/tmpenb6zx5t/rndArray_node".
190204-03:48:22,946 nipype.workflow INFO:
	 [Node] Running "rndArray_node" ("nipype.interfaces.utility.wrappers.Function")
190204-03:48:22,960 nipype.workflow INFO:
	 [Node] Finished "rndArray_node".

random_array = [[0.69606048 0.48000143 0.17902162]
 [0.10007322 0.16357615 0.83558744]
 [0.31067914 0.44169312 0.91215212]]



Now, let's see what happends if we move the import of `random` outside the scope of `get_random_array`:

In [17]:
from nipype import Node, Function

# Import random function
from numpy.random import random


# Create the Function object
def get_random_array(array_shape):

    return random(array_shape)

# Create Function Node that executes get_random_array
rndArray = Node(Function(input_names=["array_shape"],
                         output_names=["random_array"],
                         function=get_random_array),
                name='rndArray_node')

# Specify the array_shape of the random array
rndArray.inputs.array_shape = (3, 3)

# Run node
try:
    rndArray.run()
except(NameError) as err:
    print("NameError:", err)
else:
    raise

190204-03:51:13,348 nipype.workflow INFO:
	 [Node] Setting-up "rndArray_node" in "/tmp/tmpety5crao/rndArray_node".
190204-03:51:13,360 nipype.workflow INFO:
	 [Node] Running "rndArray_node" ("nipype.interfaces.utility.wrappers.Function")
190204-03:51:13,370 nipype.workflow WARNING:
	 [Node] Error on "rndArray_node" (/tmp/tmpety5crao/rndArray_node)
NameError: name 'random' is not defined
